<a href="https://colab.research.google.com/github/rawanMahmoud81/Machine-Learning-Data-analysis/blob/main/tomata_fresh_rotten.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rawanmahmoud1","key":"bd60c0609b815792f39ae56096d1884b"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d filipemonteir/fresh-and-rotten-fruits-and-vegetables


Dataset URL: https://www.kaggle.com/datasets/filipemonteir/fresh-and-rotten-fruits-and-vegetables
License(s): unknown
fresh-and-rotten-fruits-and-vegetables.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!pip install kaggle


In [ ]:
import zipfile
import os

zip_path = "/content/fresh-and-rotten-fruits-and-vegetables.zip"
extract_path = "/content/tomato_clf"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted successfully!")


Dataset extracted successfully!


In [ ]:
import os
import shutil

checkpoints_path = "/content/data/.ipynb_checkpoints"
if os.path.exists(checkpoints_path):
    shutil.rmtree(checkpoints_path)


In [ ]:
import tensorflow as tf
IMG_SIZE = (160, 160)
BATCH_SIZE = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "data/train",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='binary'
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "data/val",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='binary'
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "data/test",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='binary'
)


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:
import os

def count_images_per_class(directory):
    for class_name in os.listdir(directory):
        class_path = os.path.join(directory, class_name)
        if os.path.isdir(class_path):
            count = len([f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
            print(f"{class_name}: {count} images")

print("Training set:")
count_images_per_class("/content/data/train")

print("\Testing set:")
count_images_per_class("/content/data/test")

print("Val set:")
count_images_per_class("/content/data/val")



Training set:
FreshTomato: 842 images
RottenTomato: 837 images
\Testing set:
FreshTomato: 240 images
RottenTomato: 238 images
Val set:
FreshTomato: 120 images
RottenTomato: 119 images


<>:13: SyntaxWarning: invalid escape sequence '\T'
<>:13: SyntaxWarning: invalid escape sequence '\T'
/tmp/ipython-input-2735293282.py:13: SyntaxWarning: invalid escape sequence '\T'
  print("\Testing set:")


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

In [ ]:
IMG_SIZE = (160, 160)

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
])

base_model = MobileNetV2(input_shape=IMG_SIZE + (3,),
                         include_top=False,
                         weights='imagenet')
base_model.trainable = False



In [ ]:
inputs = layers.Input(shape=IMG_SIZE + (3,))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(128, activation='relu')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model = models.Model(inputs, outputs)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input


In [ ]:

IMG_SIZE = (224, 224)

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
])

base_model = EfficientNetB0(input_shape=IMG_SIZE + (3,),
                            include_top=False,
                            weights='imagenet')
base_model.trainable = False


In [ ]:

inputs = layers.Input(shape=IMG_SIZE + (3,))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(128, activation='relu')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model = models.Model(inputs, outputs)
